**setup**

In [ ]:
import glob
import numpy as np


In [ ]:
basepath='/reg/neh/home/zimmf/simulation'                            #absolute path to python files
scratchpath='/reg/d/psdm/amo/amolr9316/scratch/zimmf/sphere2/'          #path to simulation results
files=glob.glob("%s/sphere1-??.npz"%scratchpath)                       #simulation results

In [ ]:
photons=[1e4,1e5] #number of photons used in the simulation
bins=[2,0]        #binning by a factor of 2^n in each direction

**show input files**

In [ ]:
N=0
for f in files:
    s=np.load(f)['settings'][0]
    print (f,s)
    N=N+s.Nimg
print('found %i files with a total of %i images'%(len(files),N))

**phase 1** (calculation of the correlations for each simulated image)

In [ ]:
from collections import deque
q=deque()
for b in bins:
    for p in photons:
        for f in files:
            cmd='%s/pythonwrapper.sh %s/recongrid.py -b %i -p 4 -N %i %s'%(basepath,basepath,b,p,f)
            q.append(cmd)

In [ ]:
print('will submit %i jobs'%len(q))

In [ ]:
import subprocess
for c in q: 
    proc=subprocess.call('/usr/local/bin/bsub -q psanaq -n 8 -o log/recongrid%%J.out %s'%c,shell=True)          
    print (c,proc)
print ('\n submitted.')

# WAIT UNTIL ALL JOBS ARE DONE

In [ ]:
%%bash
echo "running jobs:"
/usr/local/bin/bjobs -w|grep -o "recongrid.*npz"

**phase 2 - merging, normalisation and creating final result**

In [ ]:
#get outputfiles of phase 1
suffix=['' if b==0 else '-b%i'%b for b in bins]
files2=[]
for s in suffix:
    for Np in photons:
        tmp=[]
        for f in files:
            tmp.append(glob.glob("%s%s-%i-out.npz"%(f,s,Np))[0])
        files2.append(tmp)

In [ ]:
print('found %i different settings'%len(files2))
if not all([len(files2[0])==len(s) for s in files2]):
    print('different number of intermediate files found. phase 1 finished?')
else:
    print('with %i files each'%len(files2[0]))

In [ ]:
import subprocess
from itertools import product
import re

for i,(s, Np) in enumerate(product(suffix, photons)):
    curfiles=files2[i]
    photonsum=0
    corr=0
    settings=None
    simkeys=['a','r','rndphase','rndpos','Natoms','E','pixelsize','detz']
    reconkeys=['bin','Nphotons']
    
    for f in curfiles:
        print(f)
        res=np.load(f)
        photonsum+=res['photonsum']
        corr+=res['corr']
        
        #ugly hack to fix stupid format of the settings
        if settings==None:
            settings={k:res['simsettings'].take(0)[k] for k in simkeys}
            settings.update({k:res['reconsettings'][0][k] for k in reconkeys})
            settings['Nimg']=0
            settings['simtype']=res['simtype'].take(0)
        for key in simkeys:
            if not res['simsettings'].take(0)[key]==settings[key]:
                raise ValueError('different simsettings: '+ key)
        for key in reconkeys:
            if not res['reconsettings'][0][key]==settings[key]:
                raise ValueError('different reconsettings: '+ key)
        if not res['simtype'].take(0)==settings['simtype']:
            raise ValueError('different reconsettings: simtype')
        settings['Nimg']+=res['simsettings'].take(0)['Nimg']

    print('for parameters %s %i, there are %i files with %i images total.'%(s,Np,len(curfiles),settings['Nimg']))
    base=re.match('(.*)(-[0-9]+.npz)',f).group(1)
    allfile='%s%s-%i-allout.npz'%(base,s,Np)
    print('merging to %s'%allfile)
    np.savez_compressed(allfile,
                    corr=corr,
                    photonsum=photonsum,
                    settings=settings
                   )
    finalfile='%s%s-%i-final.npz'%(base,s,Np)
    cmd='%s/pythonwrapper.sh %s/recongrid_normalize.py %s %s'%(basepath,basepath,allfile,finalfile)
    proc=subprocess.call('/usr/local/bin/bsub -q psanaq -n 8 -o log/reconfinal_%%J.out %s'%cmd,shell=True)          
    print(cmd)
    print('submitted job to create %s\n\n'%finalfile)

In [ ]:
%%bash
echo "running jobs:"
/usr/local/bin/bjobs -w|grep -o "recongrid_normalize.*npz"